In [1]:
import pandas as pd
import scipy.stats as stats
import sqlite3
import math

In [2]:
# Kết nối SQLite 
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# Tạo bảng dữ liệu
cursor.execute("""
    CREATE TABLE car_tests (
        Day TEXT,
        Model TEXT,
        Score REAL
    )
""")

# Dữ liệu thử nghiệm
data = [
    ("Day 1", "A", 8), ("Day 1", "B", 9), ("Day 1", "C", 7),
    ("Day 2", "A", 7.5), ("Day 2", "B", 8.5), ("Day 2", "C", 7),
    ("Day 3", "A", 6), ("Day 3", "B", 7), ("Day 3", "C", 8),
    ("Day 4", "A", 7), ("Day 4", "B", 6), ("Day 4", "C", 5)
]

# Chèn dữ liệu vào bảng
cursor.executemany("INSERT INTO car_tests VALUES (?, ?, ?)", data)
conn.commit()
# Truy vấn dữ liệu từ SQLite
query = "SELECT Day, Model, Score FROM car_tests"
df = pd.read_sql_query(query, conn)

# Chuyển dữ liệu thành bảng tần suất 
table = pd.pivot_table(df, values="Score", index="Day", columns="Model", aggfunc="sum")
# Kiểm định Chi-square
chi2_stat, p_value, dof, expected = stats.chi2_contingency(table)

# In kết quả
print(f"Chi-square Statistic: {chi2_stat}")
print(f"P-value: {p_value}")
print(f"Degrees of Freedom: {dof}")
print("Expected Frequencies:")
print(pd.DataFrame(expected, index=table.index, columns=table.columns))
# Kiểm tra kết quả
alpha = 0.05
if p_value < alpha:
    print("Có sự khác biệt đáng kể giữa các mẫu xe (bác bỏ H0).")
else:
    print("Không có đủ bằng chứng để khẳng định sự khác biệt giữa các mẫu xe (chấp nhận H0).")

# Đóng kết nối SQLite
conn.close()

Chi-square Statistic: 0.826439691310499
P-value: 0.9913459315666546
Degrees of Freedom: 6
Expected Frequencies:
Model         A         B         C
Day                                
Day 1  7.953488  8.511628  7.534884
Day 2  7.622093  8.156977  7.220930
Day 3  6.959302  7.447674  6.593023
Day 4  5.965116  6.383721  5.651163
Không có đủ bằng chứng để khẳng định sự khác biệt giữa các mẫu xe (chấp nhận H0).
